In [1]:
import json
from pathlib import Path
import shutil

import pandas as pd
import numpy as np

from bw2io import bw2setup,remote
from bw2io.importers.io import IOImporter
from bw2io.strategies.io import tidy_tables
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba

In [2]:
print(bd.__version__)

(4, 5)


## define the database

In [3]:
pet_hiot = pd.DataFrame([[1,-2],[0,1]], # experiment setting 1 prod to 0
             index=pd.MultiIndex.from_tuples([('DK','prod1'),('DK','prod2')]),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

pet_hiot = pet_hiot.astype(pd.SparseDtype("float",0))


B = pd.DataFrame([[1,3],[1,2],[0,1],[4,0]],
             index=pd.Index(['co2_air','ch4_air','co2_accelerated_air','land_occupation']),
             columns = pd.MultiIndex.from_tuples([('DK','act1'),('DK','act2')]))

B = B.astype(pd.SparseDtype("float",0))

fd = pd.DataFrame.from_dict(
    {
    ('DK','Household'):{('DK','prod1'):-11,('DK','prod2'):-3},
    ('DK','Government'):{('DK','prod1'):-8,('DK','prod2'):-4},
    ('DK','Capital'):{('DK','prod1'):-4,('DK','prod2'):-2}
        }
    )

Bfd = pd.DataFrame([[1,3],],
index=pd.Index(['co2_air',]),
columns = pd.MultiIndex.from_tuples([('DK','Household'),
('DK','Government')])).astype(pd.SparseDtype("float",0))

In [4]:
print(pet_hiot)
print(B)
print(fd)
print(Bfd)

           DK     
         act1 act2
DK prod1  1.0 -2.0
   prod2    0  1.0
                      DK     
                    act1 act2
co2_air              1.0  3.0
ch4_air              1.0  2.0
co2_accelerated_air    0  1.0
land_occupation      4.0    0
                DK                   
         Household Government Capital
DK prod1       -11         -8      -4
   prod2        -3         -4      -2
               DK           
        Household Government
co2_air       1.0        3.0


In [5]:
pfd = pd.DataFrame((np.eye(fd.shape[1])),index=fd.columns,columns=fd.columns)
fd_total = pd.concat([fd,pfd])
fd_total = fd_total.astype(pd.SparseDtype("float",0))

In [6]:
extended_hiot = pd.concat([pet_hiot,fd_total],axis=1).fillna(0)
extended_B = pd.concat([B,Bfd],axis=1).fillna(0)

In [7]:
print(extended_hiot)

                DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0


In [8]:
print(extended_B)

                      DK                          
                    act1 act2 Household Government
co2_air              1.0  3.0       1.0        3.0
ch4_air              1.0  2.0       0.0        0.0
co2_accelerated_air    0  1.0       0.0        0.0
land_occupation      4.0    0       0.0        0.0


In [9]:
path_to_intermediate = (Path.cwd()/'results')
path_to_intermediate.mkdir(exist_ok=True)

In [10]:
metadata_dict = {'prod1':{'unit':'kg','name':'product 1'},
                 'prod2':{'unit':'kg','name':'product 2'},
                 'Household':{'unit':'unit','name':'the household'},
                 'Government':{'unit':'unit','name':'the government'},
                 'Capital':{'unit':'unit','name':'capital investments'},

                 'co2_air':{'unit':'ton', # not standard units
                            'name':'carbon dioxide',
                            'compartment':('air',)},
                 'ch4_air':{'unit':'kg',
                            'name':'methane',
                            'compartment':('air',)},
                 'co2_accelerated_air':{'unit':'kg', # additional biosphere flow
                                        'name':'carbon dioxide accelerated',
                                        'compartment':('air',)},
                 'land_occupation':{'unit':'hectare * year', # non standard composite unit
                                    'name':'land occupation',
                                    'compartment':('natural resource', 'land')}
                }

with open(path_to_intermediate/'io_metadata.json', 'w') as fp:
    json.dump(metadata_dict, fp,indent=4)

In [11]:
tidy_tables(extended_hiot,extended_B,path_to_intermediate)

In [12]:
#bd.projects.set_current('test_io_importer','test_io_importer')

In [13]:
remote.install_project('ecoinvent-3.10-biosphere','test_io_importer',
                       overwrite_existing=False)

Restoring project backup archive - this could take a few minutes...
Restored project: test_io_importer


'test_io_importer'

In [18]:
bd.projects.set_current('test_io_importer')

17:53:01 [info     ] Applying automatic update: 4.0 database search directories FTS5
17:53:01 [info     ] Reindexing database ecoinvent-3.10-biosphere


In [19]:
try:
    del bd.databases['pet_io_db biosphere']
    del bd.databases['pet_io_db']
    bd.Database('pet_io_db').delete(warn=False)
    bd.Database('pet_io_db biosphere').delete(warn=False)
except KeyError:
    print('db not there')

db not there


In [13]:
bd.databases

Databases dictionary with 0 objects

These include the links between the database being imported and the biosphere3 database. 

For those that do not have a correspondence, a new elementary flow will be created in an additional biosphere db

In [14]:
biosphere_mapping = {
    'co2_air':'349b29d1-3e58-4c66-98b9-9d1a076efd2e',
    'ch4_air':'0795345f-c7ae-410c-ad25-1845784c75f5',
    'land_occupation':'c7cb5880-4219-4051-9357-10fdd08c6f2b',
    }

In [17]:
pet_example = IOImporter(path_to_intermediate,'pet_io_db',b3mapping=dict())
pet_example.apply_strategies()

In [19]:
pet_example.write_database()

no extra biosphere flows added


100%|██████████| 5/5 [00:00<00:00, 2398.94it/s]

16:00:14+0200 [info     ] Vacuuming database            
16:00:15+0200 [info     ] Starting IO table write       
16:00:15+0200 [info     ] Adding technosphere matrix    
16:00:15+0200 [info     ] Adding biosphere matrix       


KeyError: 'ch4_air'

## test

In [23]:
test_db = bd.Database('pet_io_db')

In [24]:
act1 = test_db.get(code='act1|DK')
act2 = test_db.get(code='act2|DK')

In [25]:
# it should have CO2 emissions close to 1 ton and land use of 40000 m2 * year
for e in act1.biosphere():
    print(e)

Exchange: 40000.0 square meter-year 'Occupation, unspecified' (square meter-year, None, ('natural resource', 'land')) to 'act1' (kilogram, DK, None)>
Exchange: 907.1847534179688 kilogram 'Carbon dioxide, fossil' (kilogram, None, ('air',)) to 'act1' (kilogram, DK, None)>
Exchange: 1.0 kilogram 'Methane, fossil' (kilogram, None, ('air',)) to 'act1' (kilogram, DK, None)>


In [26]:
ipcc_2021 = ('IPCC 2021', 'climate change', 'global warming potential (GWP100)')

In [27]:
test_lca = bc.LCA({act2:1},ipcc_2021)

In [28]:
test_lca.lci()
test_lca.lcia()
test_lca.score

4655.12370300293

In [29]:
extended_hiot

DK                                  
              act1 act2 Household Government Capital
DK prod1       1.0 -2.0     -11.0       -8.0    -4.0
   prod2         0  1.0      -3.0       -4.0    -2.0
   Household   0.0  0.0       1.0          0       0
   Government  0.0  0.0         0        1.0       0
   Capital     0.0  0.0         0          0     1.0

In [30]:
test_lca.technosphere_matrix.todense()

matrix([[  1.,  -2., -11.,  -8.,  -4.],
        [  0.,   1.,  -3.,  -4.,  -2.],
        [  0.,   0.,   1.,   0.,   0.],
        [  0.,   0.,   0.,   1.,   0.],
        [  0.,   0.,   0.,   0.,   1.]])

In [31]:
pd.DataFrame(test_lca.biosphere_matrix.todense())

,0,1,2,3,4
0,40000.000000,0.000000,0.000000,0.000000,0.0
1,907.184753,2721.554199,907.184753,2721.554199,0.0
2,1.000000,2.000000,0.000000,0.000000,0.0
3,0.000000,1.000000,0.000000,0.000000,0.0


In [32]:
ba.print_recursive_calculation(act2,ipcc_2021,use_matrix_values=True)

Fraction of score | Absolute score | Amount | Activity
0001 |  4655 |     1 | 'act2' (kilogram, DK, None)
  0.403 |  1874 |     2 | 'act1' (kilogram, DK, None)


## compatibility with bw_aggregation 

In [30]:
test_db.backend

'iotable'

In [31]:
import bw_aggregation

In [32]:
bw_aggregation.AggregatedDatabase.convert_existing(test_db.name)

Starting inventory calculation. Please be patient, we have 5 processes to calculate.


In [33]:
bd.Database('pet_io_db')

Brightway2 AggregatedDatabase: pet_io_db

In [34]:
bd.Database('pet_io_db').use_aggregated(False)

In [35]:
act2 = bd.Database('pet_io_db').get(code='act2|DK')

In [36]:
ba.print_recursive_calculation(act2,ipcc_2021)

Fraction of score | Absolute score | Amount | Activity
0001 |  4655 |     1 | 'act2' (kilogram, DK, None)


In [37]:
with bw_aggregation.AggregationContext(False):

    ba.print_recursive_calculation(act2,ipcc_2021)

Fraction of score | Absolute score | Amount | Activity
0001 |  4655 |     1 | 'act2' (kilogram, DK, None)


In [38]:
lca = act2.lca(ipcc_2021)

In [41]:
# if true, then it is not aggregated..
'col_code' in lca.to_dataframe().columns

True

exchanges are no longer there

In [45]:
list(act2.exchanges())

[]